In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [4]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', -1)
get_ipython().magic(u'matplotlib inline') # same as %matplotlib inline

In [3]:
import seaborn as sns
sns.set_style("whitegrid")

# Useful functions

In [6]:
def explode(df, lst_cols, fill_value=-1):
    """
    Partial Source: https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows/40449726#40449726
    https://stackoverflow.com/questions/30885005/pandas-series-of-lists-to-one-series
    """
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        lens.fillna(1, inplace=True)
        lens = lens.map(np.int64)
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:df[col].fillna(fill_value).apply(pd.Series).stack().reset_index(drop=True) for col in lst_cols}) \
          .loc[:, df.columns]

In [16]:
df = pd.DataFrame({'col1': [1, 2, 3, 4], 'col2': [['A', 'a'], ['B', 'b'], ['C', 'c'], None]})

In [17]:
explode(df, 'col2')

,col1,col2
0,1,A
1,1,a
2,2,B
3,2,b
4,3,C
5,3,c
6,4,-1


## Expand column width when saving to an excel

In [ ]:
writer = pd.ExcelWriter(filename, engine='xlsxwriter')
sheetname = 'data'
df.to_excel(writer, sheet_name=sheetname, index=False)  # send df to writer
worksheet = writer.sheets[sheetname]  # pull worksheet object
for idx, col in enumerate(df):  # loop through all columns
    series = df[col]
    max_len = max((
        series.astype(str).map(len).max(),  # len of largest item
        len(str(series.name))  # len of column name/header
        ))   # adding a little extra space
    worksheet.set_column(idx, idx, max_len)  # set column width
writer.save()

# Data exploration

An awesome library for visualizing pandas dataframe is [qgrid](https://github.com/quantopian/qgrid). it helps in data explorations


In [4]:
import qgrid
randn = np.random.randn
df_types = pd.DataFrame({
    'A' : pd.Series(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08', '2013-01-09'],index=list(range(9)),dtype='datetime64[ns]'),
    'B' : pd.Series(randn(9),index=list(range(9)),dtype='float32'),
    'C' : pd.Categorical(["washington", "adams", "washington", "madison", "lincoln","jefferson", "hamilton", "roosevelt", "kennedy"]),
    'D' : ["foo", "bar", "buzz", "bippity","boppity", "foo", "foo", "bar", "zoo"] })
df_types['E'] = df_types['D'] == 'foo'
qgrid_widget = qgrid.QgridWidget(df=df_types, show_toolbar=True)
qgrid_widget

UWdyaWRXaWRnZXQoZ3JpZF9vcHRpb25zPXsnaGlnaGxpZ2h0U2VsZWN0ZWRSb3cnOiBUcnVlLCAnZnVsbFdpZHRoUm93cyc6IFRydWUsICdyb3dIZWlnaHQnOiAyOCwgJ2VuYWJsZUNvbHVtblLigKY=


If you make any sorting/filtering changes, or edit the grid by double clicking, you can retrieve a copy of your DataFrame which reflects these changes by calling get_changed_df on the QgridWidget instance returned by show_grid.

In [5]:
qgrid_widget.get_changed_df()

,A,B,C,D,E
2,2013-01-03,-0.736818,washington,buzz,False
0,2013-01-01,0.030765,washington,foo,True
7,2013-01-08,-1.355409,roosevelt,bar,False
3,2013-01-04,0.689274,madison,bippity,False
4,2013-01-05,0.367802,lincoln,boppity,False
8,2013-01-09,0.544976,kennedy,zoo,False
5,2013-01-06,-2.361572,jefferson,foo,True
6,2013-01-07,-0.785215,hamilton,foo,True
1,2013-01-02,1.018985,adams,bar,False
